In [1]:
import os
import json
import shutil, glob

# How to use this notebook

- this notebook expects a data folder in its current path
- in the data folder `pathEvaluator.json` is required
- create a pcap folder in the data folder and past the `.pcap/.raw` file along with its `key.log` file
- rename the next cell variables accordingly 

In [2]:
pcap_file ="./data/pcap/chrome-sat.raw"
key_log = "./data/pcap/key.log"

# Path Evaluator

In [3]:
# Loading the pathEvaluator json which contain the data regarding the question and choices
with open('./data/pathEvaluator.json', 'r') as f:
    main_data = json.load(f)

In [4]:
# Question data subset of main_data containing only info about question
question_data = main_data['jsonGraph']['videos']['80988062']['interactiveVideoMoments']['value']["choicePointNavigatorMetadata"]["choicePointsMetadata"]["choicePoints"]  

# choice data subset of main_data containing only info about choices
choice_data = main_data['jsonGraph']['videos']['80988062']['interactiveVideoMoments']['value']["momentsBySegment"]

In [5]:
# bs_data relates parent_id, chocie_id, question, chocice text and default option
bs_data = {}
# stores the choice id and the corresponding text
choice_text = {}

In [6]:
# returns the question for a given patent_id
# uses parent_if to find the question
# requires question data to be declared in the global scope
def find_ques(parent_id):
    return question_data[parent_id]["description"]
    
# example 
# print(find_ques(parent_id="3X"))

In [7]:
# add the relevant data (relates parent_id, chocie_id, question, chocice text and default option) to bs_data from choice_data
# arg : parent_id (id of the previous choice)
# expects question_data and choce_data is present in the global scope
# add the id and corresponding choice text to the choice_text 

def gen_data(parent_id):
    temp = {}
    # Some options may not have any Question 
    # example: single options choice are mostlikely not to have any question
    try:
        temp['question'] = find_ques(parent_id)
    except:
        # do nothing
        pass
        
    data = choice_data[parent_id]
    for items in data:
        if "choices" in items:
            temp["defaultChoiceIndex"] = items["defaultChoiceIndex"]
            # some choices don't have any text
            # example: there is a choice point where the user need to type the phone no 
            # correct phone no will be one choice and the wrong one will be the other.
            try:
                temp["choices"] = [{"id": item["id"], "text": item["text"], "segmentId": item["sg" if "sg" in item else "segmentId"]} for item in items["choices"]]
                
            except:
                # do something to add info about the phone no.
                # if ph no is available add else add xxx
                # temp[choice] = [{},{}]
                temp["choices"] = []
                for item in items["choices"]:
                    temp_dict = {}
                    temp_dict["id"] = item["id"]
                    if "segmentId" in item:
                        temp_dict["segmentId"] = item["segmentId"]
                    elif "sg" in item:
                        temp_dict["segmentId"] = item["sg"]
                    elif "trackingInfo" in items:
                        temp_dict["segmentId"] = item["trackingInfo"]["segmentId"]
                    if "text" in item:
                        temp_dict["text"] = item["text"]
                    elif "code" in item:
                        temp_dict["code"] = item["code"]
                    else:
                        temp_dict["code"] = "xxx"
                    temp["choices"].append(temp_dict)
            # adding the choice id and text to choice_text
            for choice in temp["choices"]:
                if choice["id"] not in choice_text:
                    choice_text[choice["segmentId"]] = choice['text' if "text" in choice else "code"]
                
    bs_data[parent_id]=temp


In [8]:
# returns the default choice for a given parent id
# expects the bs_data (created already) is available in the global scope
# bs_data relates parent_id, chocie_id, question, chocice text and default option

def find_default(parent_id):
    return bs_data[parent_id]["choices"][bs_data[parent_id]["defaultChoiceIndex"]]

    

In [9]:
def get_choice_text(choice_id):
    with open('./data/choice_text.json', 'r') as f:
        data = json.load(f)
    return(data[choice_id])

In [10]:
def get_choices_list(parent_id):
    with open('./data/bs_data.json', 'r') as f:
        data = json.load(f)
    return data[parent_id]["choices"]

# example
# get_choices_list("1H")

In [11]:
# storing all id's
parent_ids=choice_data.keys();

In [12]:
# generating data for all the id's
for parent_id in parent_ids:
#     print(parent_id)
    gen_data(parent_id)
    
    
blank_keys = []
for key, data in bs_data.items():
    if len(data) == 0:
        blank_keys.append(key) 
for key in blank_keys:
    del bs_data[key]
len(bs_data)
    
# storing main data
with open('./data/bs_data.json', 'w') as json_file:
  json.dump(bs_data, json_file)

# saving the choice text
with open('./data/choice_text.json', 'w') as json_file:
  json.dump(choice_text, json_file)

# Data cl2

- first, find what is selected
- second, find if it was default or non-default

In [13]:
# stores the result
selection_data = {}

# uncomment the print statemensts to decode
def decode_cl2(cl2_path="./data/data_cl2.json"):
#     print("Processing : {}".format(cl2_path))
    with open(cl2_path, 'r') as f:
        data_cl2 = json.load(f)
    
    selected_coice_sid = ""
    parent_id = ""
    reverseDeltas = data_cl2["reverseDeltas"]
    currentState = data_cl2["currentState"]
    temp = {}
    
    for key, items in currentState.items():
        if "view" in items:
            if items["view"] == "ikoChoicePoint":
                parent_id = items["trackingInfo"]["momentId"].split("_")[1]
                given_choices= items["trackingInfo"]["choices"]
                break
    
    for items in reverseDeltas:
        for item in items:
            if type(item) == dict and "view" in item:
                if item["view"] == "ikoChoicePointOption":
                    selected_coice_sid = item["trackingInfo"]["segmentId"]
                elif item["view"] == "ikoChoicePoint":
                    parent_id = item["trackingInfo"]["momentId"].split("_")[1]
                    given_choices= item["trackingInfo"]["choices"]

        
    if selected_coice_sid != "" and parent_id != "":
        try:
            temp["id"] = parent_id 
            temp["question"] = find_ques(parent_id) 
#             print("Question Id: {}\t text: {}".format(parent_id,find_ques(parent_id)))
        except:
            temp["error"] = "No queston present" 
#             print("No Question present for this choice! ")

        if len(given_choices) == 2:
            try:
#                 print("Selected choice Id: {}\t text: {}".format(selected_coice_sid,get_choice_text(selected_coice_sid)))
                temp["sid"] = selected_coice_sid 
                temp["ctext"] = get_choice_text(selected_coice_sid) 
            except:
                # May Crash Queck Fix.
#                 print("Opsee! error finding selected choice.. trying again!")
                found_choice = ""
                for choice in given_choices:
                    if choice["segmentId"] == selected_coice_sid:
                        selected_coice_id = choice["id"]
                        for ch in get_choices_list(parent_id):
                            if ch["id"] == selected_coice_id:
                                found_choice = ch["text" if "text" in ch else "code"]
#                 print("Selected choice Id: {}\t text: {}".format(selected_coice_sid,found_choice))
                temp["sid"] = selected_coice_sid 
                temp["ctext"] = found_choice 
            
#             print("Default chocie: "+str(find_default(parent_id)))
            temp["dchoice"] = find_default(parent_id)
            selection_data[parent_id]=temp

In [14]:
# exporting the http objects
dest_folder = "data/trash" 
export_obj_cmd ="tshark -nr {} -o tls.keylog_file:{} --export-objects \"http,{}\"".format(pcap_file, key_log, dest_folder)
os.system(export_obj_cmd)

# moving all cl2 files from trash to cl2 folder
files = glob.glob('./data/trash/cl2*')
for f in files:
    src = f
    dst = f.replace("trash", "cl2")
    shutil.move(src,dst)


In [15]:
# decode each cl2
selection_data = {}
for cl2 in glob.glob('./data/cl2/cl2*'):
    decode_cl2(cl2)

In [16]:
# printing the result
print(json.dumps(selection_data, indent=2))

{
  "1A": {
    "id": "1A",
    "question": "Which Cereal?",
    "sid": "1D",
    "ctext": "FROSTIES",
    "dchoice": {
      "id": "1E",
      "text": "SUGAR PUFFS",
      "segmentId": "1E"
    }
  },
  "1D": {
    "id": "1D",
    "question": "Which Tape?",
    "sid": "1G",
    "ctext": "NOW 2",
    "dchoice": {
      "id": "1H",
      "text": "THOMPSON TWINS",
      "segmentId": "1H"
    }
  },
  "1G": {
    "id": "1G",
    "question": "Job Offer",
    "sid": "1Qnw",
    "ctext": "REFUSE",
    "dchoice": {
      "id": "nsg-8AChoice",
      "text": "ACCEPT",
      "segmentId": "8AChoice"
    }
  },
  "1Qnw": {
    "id": "1Qnw",
    "question": "Talk About Mum?",
    "sid": "2B",
    "ctext": "YES",
    "dchoice": {
      "id": "nsg-2BChoice",
      "text": "YES",
      "segmentId": "2BChoice"
    }
  },
  "2B": {
    "id": "2B",
    "question": "Which Record?",
    "sid": "1R",
    "ctext": "PHAEDRA",
    "dchoice": {
      "id": "1R",
      "text": "PHAEDRA",
      "segmentId": "1R"


In [17]:
# saving the output in a json file
with open('./data/user_choice.json', 'w') as json_file:
  json.dump(selection_data, json_file)